In [ ]:
import itertools
import functools
import numpy as np
import random
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.collections import PathCollection

from gan import Gan, Data
import pltanimation as plta

NUM_SAMPLES=100

dm = 20   # discriminator granularity
ds = 0.025 # discriminator square size
tics = [(t+0.5)/dm for t in range(-dm, dm, 1)]
tics_xy = np.array(list(itertools.product(tics, tics)))

def animation_init(data, fig):    
  ax = fig.add_subplot(1,1,1)
  ax.set_title("Empty plot")
  #plot1 = ax.plot([], [])[0]
  real = ax.scatter(data.real_x[:,0], data.real_x[:,1], label='real', s=10)
  generator_out = ax.scatter([], [], label='generator', s=5)
  box = ax.add_patch(Rectangle((0, 0), 0.5, 0.5, 
        linewidth=0, edgecolor='b', facecolor=(0.6,1.0,0.6), zorder=-1))
    
  boxes = []
  
  for x, y in tics_xy:
    boxes.append(ax.add_patch(Rectangle((x-ds, y-ds), 2*ds, 2*ds, 
      linewidth=0, edgecolor='b', facecolor=(0.6,1.0,0.6), zorder=-1)))
    
  """
    
    tics_xy = np.array(list(itertools.product(tics, tics)))
    tics_pred = discriminator.predict(tics_xy)
    dis_items = np.concatenate((tics_xy, tics_pred), axis=-1)
    dis_items = dis_items[(dis_items[:,2] >= 0.5)]
    for x, y, pred in dis_items:
      plt.gca().add_patch(plt.Rectangle((x-ds, y-ds), 2*ds, 2*ds, 
        linewidth=0, edgecolor='b',facecolor=(0.6,1.0,0.6),zorder=-1))
  """
    
  ax.legend(loc='upper left')
  ax.grid(True)
  ax.set_xlim((-1,1))
  ax.set_ylim((-1,1))
  return [generator_out] + boxes

def animation_callback(ani, gan_model, generator, discriminator, generator_output, discriminator_output, score):

    box_updates = []
    for x, y in tics_xy:
      out = discriminator.predict([[x, y]])[0]
      if (out >= 0.5):
          box_updates.append((Rectangle.set_facecolor, (0.6,1.0,0.6)))
      else:
          box_updates.append((Rectangle.set_facecolor, (1,1,1))) 

    ani.add_frame([
        (PathCollection.set_offsets, generator_output)        
    ] + box_updates)

def gen_data(num_samples):
  noise = np.array([random.normalvariate(0, 0.4) for _ in range(num_samples)])
  x1 = np.array([random.random() * 2 - 1 for _ in range(num_samples)])
  x2 = np.array((np.power(x1*2, 3) + np.sin(x1*12 - 2)*0.7 + noise)/7)
  y = np.full(shape=(num_samples, 1), fill_value=1)
  x1 = x1.reshape(-1,1)
  x2 = x2.reshape(-1,1)    
  return np.concatenate((x1, x2), axis=-1), y

if __name__ == '__main__':
    
  real_x, real_y = gen_data(NUM_SAMPLES)
  real_x_val, real_y_val = gen_data(NUM_SAMPLES//4)
  data = Data(real_x=real_x, real_y=real_y, real_x_val=real_x_val, real_y_val=real_y_val)
  
  ani = plta.PltAnimation(functools.partial(animation_init, data), figsize=(10, 10)) 
    
  latent_shape = (5,)
  data_shape = (2,) # data shape
  num_samples = 40

  # Get real words for training.
  latent_space = tf.keras.layers.Input(shape=latent_shape)

  # Using Leaky RELU because of tip https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/.

  # Discriminator input is (n, 21, 27) output is (n, 1)
  discriminator = tf.keras.models.Sequential(name="discriminator")
  discriminator.add(tf.keras.layers.Dense(name='dhidden1', units=50, use_bias=True,
                                          activation=tf.keras.layers.LeakyReLU(alpha=0.3), input_shape=data_shape))
  discriminator.add(tf.keras.layers.Dense(name='dhidden2', units=50, use_bias=True,
                                          activation=tf.keras.layers.LeakyReLU(alpha=0.3)))
  discriminator.add(tf.keras.layers.Dense(name='dhidden3', units=15, use_bias=True,
                                          activation=tf.keras.layers.LeakyReLU(alpha=0.3)))
  discriminator.add(tf.keras.layers.Dense(name='doutput', units=1, use_bias=True, activation='tanh'))

  #discriminator.add(tf.keras.layers.Dense(name='dhidden1', units=50, use_bias=True, activation='tanh', input_shape=data_shape))
  #discriminator.add(tf.keras.layers.Dense(name='dhidden2', units=50, use_bias=True, activation='tanh'))
  #discriminator.add(tf.keras.layers.Dense(name='dhidden3', units=15, use_bias=True, activation='tanh'))
  #discriminator.add(tf.keras.layers.Dense(name='doutput', units=1, use_bias=True, activation='linear'))
  discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='binary_crossentropy', metrics=['binary_accuracy'])

  # Generator.
  discriminator.trainable = False

  generator = tf.keras.models.Sequential(name="generator")
  generator.add(tf.keras.layers.Dense(name='ghidden1', units=15, use_bias=True, activation='tanh', input_shape=latent_shape))
  generator.add(tf.keras.layers.Dense(name='ghidden2', units=15, use_bias=True, activation='tanh'))
  generator.add(tf.keras.layers.Dense(name='gout', units=2, use_bias=True, activation='linear'))

  gan_model = tf.keras.models.Model(latent_space, discriminator(generator(latent_space)))
  gan_model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='mse', metrics=['binary_accuracy'])

  gan = Gan(gan_model, data)
  gan.train(iterations=100, epochs_per_round=1, num_samples=1000, train_discriminator_only=True, verbose=1, callback=functools.partial(animation_callback, ani))
  

---------- Round 0 predictions: 0.05384747311472893 ----------
Mean score is 0.05384747311472893 -- training discriminator.
Train on 1100 samples, validate on 275 samples
1100/1100 [==============================] - 0s 304us/sample - loss: 0.2992 - binary_accuracy: 0.9091 - val_loss: 0.2095 - val_binary_accuracy: 0.9091
---------- Round 1 predictions: 0.009539589285850525 ----------
Mean score is 0.009539589285850525 -- training discriminator.
Train on 1100 samples, validate on 275 samples
1100/1100 [==============================] - 0s 54us/sample - loss: 0.2012 - binary_accuracy: 0.9091 - val_loss: 0.1829 - val_binary_accuracy: 0.9091
---------- Round 2 predictions: -0.021792106330394745 ----------
Mean score is -0.021792106330394745 -- training discriminator.
Train on 1100 samples, validate on 275 samples
1100/1100 [==============================] - 0s 57us/sample - loss: 0.1799 - binary_accuracy: 0.9091 - val_loss: 0.1751 - val_binary_accuracy: 0.9091
---------- Round 3 predictions

In [ ]:
ani.as_html()